# Вычисление среднего вектора набора данных

Для представления текста в виде вектора используется бинарный вектор, для нормализации слов используется Yandex MyStem, 

# Модель новости

In [1]:
import json
import numpy as np
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer

class News:
    def __init__(self, id, date, annotation, title, content, url):
        self.id = id
        self.date = date
        self.annotation = annotation
        self.title = title
        self.content = content
        self.url = url
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

# Загружаем новости

In [2]:
news = []
with open('/data/kasandra/news-1000.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

list(map(lambda w: w.title, news[:5]))

['На Алтае заработала первая в РФ система контроля космического пространства',
 'ЛНР: Действия украинских диверсантов — одна из версий покушения на Плотницкого',
 'Bild не публикует результаты России в медальном зачёте Олимпиады',
 'Глава ЛНР связал покушение на свою жизнь с украинскими спецслужбами',
 'Мэра города в США арестовали за попытку расплатиться наркотиками за секс с мужчинами']

# Загружаем стоп слова

In [3]:
stopwords = []

with open('../main/python/res/stopwords.txt', mode="r", encoding="utf8") as file:
    for line in file:
        stopwords.append(line.replace('\n', ''))

# Подключаем Yandex MyStem для нормализации текста

In [4]:
mystem = Mystem(mystem_bin='/data/mystem/mystem',
                entire_input=False)

# Оставляем только русские слова
r = re.compile('^[А-ЯЙа-яй]*$')

def lemmatize(text):
    return list(filter(r.match, mystem.lemmatize(text)))

## Преобразуем текст в вектор с помощью CountVectorizer

In [5]:
vectorizer = CountVectorizer(tokenizer=lemmatize, stop_words=stopwords, binary=True)

bin_matrix = vectorizer.fit_transform(map(lambda n: n.content, news))

# Вычисление среднего вектора

[Centroid](https://en.wikipedia.org/wiki/Centroid#Of_a_finite_set_of_points)

In [6]:
def mean_vector(bin_matrix):
    mean = bin_matrix[0].toarray()
    for v in bin_matrix[1:]:
        mean = mean + v.toarray()
    return mean / bin_matrix.shape[0]

def binary(val, n):
    if val < n:
        return 0
    else:
        return 1

binary_vector = np.vectorize(binary)

In [7]:
mean = mean_vector(bin_matrix)
names = vectorizer.get_feature_names()
wd = zip(names, mean[0])

list(filter(lambda x: x[1] > 0.2, wd))

[('авг', 0.29499999999999998),
 ('важный', 0.30299999999999999),
 ('время', 0.26100000000000001),
 ('год', 0.42299999999999999),
 ('заявлять', 0.29299999999999998),
 ('кнопка', 0.28399999999999997),
 ('курс', 0.28699999999999998),
 ('любой', 0.33200000000000002),
 ('москва', 0.34300000000000003),
 ('мочь', 0.28899999999999998),
 ('нажимать', 0.28199999999999997),
 ('новость', 0.78300000000000003),
 ('отмечать', 0.22500000000000001),
 ('подписываться', 0.28100000000000003),
 ('проходить', 0.38100000000000001),
 ('ранее', 0.25600000000000001),
 ('российский', 0.30499999999999999),
 ('россия', 0.39700000000000002),
 ('самый', 0.39700000000000002),
 ('слово', 0.29799999999999999),
 ('сообщать', 0.39500000000000002),
 ('ссылка', 0.33800000000000002),
 ('становиться', 0.252),
 ('страна', 0.23200000000000001),
 ('устройство', 0.29299999999999998),
 ('человек', 0.30199999999999999)]

# Вычисление вектора за два дня


02.08.2016: 1470096000 - 1470182399

05.08.2016: 1470355200 - 1470441599

In [8]:
news02 = filter(lambda n: n.date >= 1470096000 and n.date <= 1470182399, news) # 02.08.2016
news05 = filter(lambda n: n.date >= 1470355200 and n.date <= 1470441599, news) # 05.08.2016

content02 = list(map(lambda n: n.content, news02))
content05 = list(map(lambda n: n.content, news05))

## Вычисляем вектора

In [9]:
vectorizer = CountVectorizer(tokenizer=lemmatize, stop_words=stopwords, binary=True)
vectorizer.fit(content02)
vectorizer.fit(content05)

bin_matrix02 = vectorizer.transform(content02)
bin_matrix05 = vectorizer.transform(content05)

# Вычисляем средние вектора

In [26]:
mean02 = mean_vector(bin_matrix02)
mean05 = mean_vector(bin_matrix05)

## Веса слов

In [24]:
max_val = 0.3
list(filter(lambda x: x[1] > max_val, zip(names, mean02[0])))

[('авг', 0.3014705882352941),
 ('важный', 0.3235294117647059),
 ('год', 0.48529411764705882),
 ('кнопка', 0.3014705882352941),
 ('курс', 0.30882352941176472),
 ('любой', 0.33088235294117646),
 ('нажимать', 0.3014705882352941),
 ('новость', 0.69117647058823528),
 ('подписываться', 0.3014705882352941),
 ('проходить', 0.38970588235294118),
 ('россия', 0.36764705882352944),
 ('самый', 0.39705882352941174),
 ('сообщать', 0.33823529411764708),
 ('ссылка', 0.375),
 ('устройство', 0.3014705882352941)]

In [25]:
max_val = 0.3
list(filter(lambda x: x[1] > max_val, zip(names, mean05[0])))

[('важный', 0.37583892617449666),
 ('год', 0.40268456375838924),
 ('кнопка', 0.36912751677852351),
 ('курс', 0.3825503355704698),
 ('любой', 0.38926174496644295),
 ('москва', 0.30201342281879195),
 ('нажимать', 0.36912751677852351),
 ('новость', 0.78523489932885904),
 ('подписываться', 0.36241610738255031),
 ('приводить', 0.3087248322147651),
 ('проходить', 0.42281879194630873),
 ('российский', 0.30201342281879195),
 ('россия', 0.34228187919463088),
 ('самый', 0.48993288590604028),
 ('слово', 0.32214765100671139),
 ('сообщать', 0.37583892617449666),
 ('ссылка', 0.40939597315436244),
 ('устройство', 0.35570469798657717)]

### Вычисляем общие слова в векторах

In [11]:
max_val = 0.1
names = vectorizer.get_feature_names()
meanBin02 = binary_vector(mean02, max_val)
meanBin05 = binary_vector(mean05, max_val)

diff = meanBin02 + meanBin05

list(filter(lambda x: x[1] == 2, zip(names, diff[0])))

[('авг', 2),
 ('август', 2),
 ('агентство', 2),
 ('важный', 2),
 ('вид', 2),
 ('владимир', 2),
 ('власть', 2),
 ('вопрос', 2),
 ('время', 2),
 ('выступать', 2),
 ('глава', 2),
 ('год', 2),
 ('данные', 2),
 ('дело', 2),
 ('день', 2),
 ('должный', 2),
 ('заявлять', 2),
 ('июль', 2),
 ('кнопка', 2),
 ('компания', 2),
 ('курс', 2),
 ('любой', 2),
 ('международный', 2),
 ('мир', 2),
 ('мнение', 2),
 ('москва', 2),
 ('мочь', 2),
 ('нажимать', 2),
 ('называть', 2),
 ('напоминать', 2),
 ('новость', 2),
 ('новый', 2),
 ('опубликовывать', 2),
 ('отмечать', 2),
 ('первый', 2),
 ('подписываться', 2),
 ('получать', 2),
 ('представитель', 2),
 ('президент', 2),
 ('приводить', 2),
 ('принимать', 2),
 ('проходить', 2),
 ('работа', 2),
 ('ранее', 2),
 ('результат', 2),
 ('решение', 2),
 ('российский', 2),
 ('россия', 2),
 ('самый', 2),
 ('сила', 2),
 ('ситуация', 2),
 ('слово', 2),
 ('случай', 2),
 ('сми', 2),
 ('смочь', 2),
 ('сообщать', 2),
 ('ссылка', 2),
 ('становиться', 2),
 ('сторона', 2),
 ('стр